# Tworzenie modelu TTS



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
## Install Coqui TTS
! pip install -U pip
! pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 11.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 289.9/289.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.9/90.9 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 84.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 97.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 24.8 MB/s eta 0:00:00
     ━━

## Przygotowanie datasetu



In [ ]:
import os

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig

output_path = "/content/drive/MyDrive/original/tts_train_dir"
if not os.path.exists(output_path):
    os.makedirs(output_path)


In [ ]:
# Download and extract LJSpeech dataset.

!wget -O $output_path/LJSpeech-1.1.tar.bz2 https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
!tar -xf $output_path/LJSpeech-1.1.tar.bz2 -C $output_path

--2023-12-06 01:36:27--  https://data.keithito.com/data/speech/LJSpeech-1.1.tar.bz2
Resolving data.keithito.com (data.keithito.com)... 24.199.73.137
Connecting to data.keithito.com (data.keithito.com)|24.199.73.137|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2748572632 (2.6G) [text/plain]
Saving to: ‘/content/drive/MyDrive/original/tts_train_dir/LJSpeech-1.1.tar.bz2’

/content/drive/MyDr 100%[===================>]   2.56G  67.8MB/s    in 37s     

2023-12-06 01:37:05 (70.1 MB/s) - ‘/content/drive/MyDrive/original/tts_train_dir/LJSpeech-1.1.tar.bz2’ saved [2748572632/2748572632]



In [ ]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=os.path.join(output_path, "LJSpeech-1.1/")
)
print(dataset_config)

BaseDatasetConfig(formatter='ljspeech', dataset_name='', path='/content/drive/MyDrive/original/tts_train_dir/LJSpeech-1.1/', meta_file_train='metadata.csv', ignored_speakers=None, language='', phonemizer='', meta_file_val='', meta_file_attn_mask='')


## Trening Modelu

Ustawienie konfiguracji


In [ ]:
# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=100,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
    save_step=1000,
)

Inicjalizacja Audio Procesora

In [ ]:
from TTS.utils.audio import AudioProcessor
ap = AudioProcessor.init_from_config(config)
# Modify sample rate if for a custom audio dataset:
# ap.sample_rate = 22050


 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log10
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:True
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:None
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:20.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:45
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:10
 | > hop_length:256
 | > win_length:1024


In [ ]:
from TTS.tts.utils.text.tokenizer import TTSTokenizer
tokenizer, config = TTSTokenizer.init_from_config(config)

In [ ]:
from TTS.tts.datasets import load_tts_samples
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

 | > Found 13100 files in /content/drive/MyDrive/original/tts_train_dir/LJSpeech-1.1


Inicjalizacja modelu

In [ ]:
from TTS.tts.models.glow_tts import GlowTTS
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

In [ ]:
from trainer import Trainer, TrainerArgs
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

 > Training Environment:
 | > Backend: Torch
 | > Mixed precision: True
 | > Precision: fp16
 | > Current device: 0
 | > Num. of GPUs: 1
 | > Num. of CPUs: 2
 | > Num. of Torch Threads: 1
 | > Torch seed: 54321
 | > Torch CUDNN: True
 | > Torch CUDNN deterministic: False
 | > Torch CUDNN benchmark: False
 | > Torch TF32 MatMul: False
 > Start Tensorboard: tensorboard --logdir=/content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000

 > Model has 28610257 parameters


### Rozpoczecie treningu




In [ ]:
trainer.fit()


 > EPOCH: 0/100
 --> /content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000


[*] Pre-computing phonemes...


  0%|          | 3/12969 [00:01<1:52:47,  1.92it/s]

ɪnstɛd əv weɪtɪŋ ðɛɹ, ɔzwɔld əpɛɹəntli wɛnt æz fɑɹ əweɪ æz hi kʊd ænd bɔɹdɪd ðə fɚst oʊk klɪf bʌs wɪt͡ʃ keɪm əlɔŋ
 [!] Character '͡' not found in the vocabulary. Discarding it.


 16%|█▌        | 2059/12969 [01:55<06:56, 26.22it/s]

...


  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.007891803979873657 (+0.0009070932865142822)
     | > avg_loss: 2.9907763302326202 (-0.32978662848472595)
     | > avg_log_mle: 0.7172440364956856 (-0.03106924146413803)
     | > avg_loss_dur: 2.27353236079216 (-0.29871734976768494)

 > BEST MODEL : /content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000/best_model_1218.pth

 > EPOCH: 3/100
 --> /content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000

 > TRAINING (2023-12-06 02:25:05) 

   --> TIME: 2023-12-06 02:25:17 -- STEP: 7/406 -- GLOBAL_STEP: 1225
     | > loss: 3.026080846786499  (3.112809113093785)
     | > log_mle: 0.7211859226226807  (0.718170131955828)
     | > loss_dur: 2.3048949241638184  (2.39463894707816)
     | > amp_scaler: 16384.0  (16384.0)
     | > grad_norm: tensor(6.0849, device='cuda:0')  (tensor(6.2423, device='cuda:0'))
     | > current_lr: 7.5e-07 
     | > step_time: 0.8326  (1.1962013925824846)
     | > 

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.006057292222976685 (-0.0018345117568969727)
     | > avg_loss: 2.8965582847595215 (-0.09421804547309875)
     | > avg_log_mle: 0.6587095484137535 (-0.05853448808193207)
     | > avg_loss_dur: 2.2378487586975098 (-0.03568360209465027)

 > BEST MODEL : /content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000/best_model_1624.pth

 > EPOCH: 4/100
 --> /content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000

 > TRAINING (2023-12-06 02:35:49) 

   --> TIME: 2023-12-06 02:35:55 -- STEP: 1/406 -- GLOBAL_STEP: 1625
     | > loss: 2.964743137359619  (2.964743137359619)
     | > log_mle: 0.6646634340286255  (0.6646634340286255)
     | > loss_dur: 2.300079584121704  (2.300079584121704)
     | > amp_scaler: 16384.0  (16384.0)
     | > grad_norm: tensor(5.2000, device='cuda:0')  (tensor(5.2000, device='cuda:0'))
     | > current_lr: 1e-06 
     | > step_time: 1.3158  (1.3157923221588135)
     | >

 | > Synthesizing test sentences.



  --> EVAL PERFORMANCE
     | > avg_loader_time: 0.010662168264389038 (+0.0046048760414123535)
     | > avg_loss: 2.5343425571918488 (-0.36221572756767273)
     | > avg_log_mle: 0.5874077007174492 (-0.07130184769630432)
     | > avg_loss_dur: 1.9469348788261414 (-0.2909138798713684)

 > BEST MODEL : /content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000/best_model_2030.pth

 > EPOCH: 5/100
 --> /content/drive/MyDrive/original/tts_train_dir/run-December-06-2023_01+44AM-0000000

 > TRAINING (2023-12-06 02:46:37) 

   --> TIME: 2023-12-06 02:47:03 -- STEP: 20/406 -- GLOBAL_STEP: 2050
     | > loss: 2.48616361618042  (2.6068166971206663)
     | > log_mle: 0.6098449230194092  (0.6022091299295426)
     | > loss_dur: 1.8763188123703003  (2.0046075642108914)
     | > amp_scaler: 16384.0  (16384.0)
     | > grad_norm: tensor(4.3723, device='cuda:0')  (tensor(4.5662, device='cuda:0'))
     | > current_lr: 1.2499999999999999e-06 
     | > step_time: 0.9191  (1.0110357

In [ ]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [ ]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [ ]:
from google.colab import files
files.download('/content/tts_train_dir/run-December-05-2023_03+58PM-0000000/config.json')
files.download('/content/tts_train_dir/run-December-05-2023_03+58PM-0000000/best_model.pth')
files.download('/content/tts_train_dir/LJSpeech-1.1/metadata.csv')

FileNotFoundError: ignored

In [ ]:
!zip -r /content/run-December-05-2023_03+58PM-0000000.zip /content/tts_train_dir/run-December-05-2023_03+58PM-0000000
from google.colab import files
files.download("/content/run-December-05-2023_03+58PM-0000000.zip")

NotImplementedError: ignored

In [ ]:
!pip install tensorboard
!tensorboard --logdir=tts_train_dir

## Testowanie modelu


In [ ]:
import glob, os
output_path = "tts_train_dir"
ckpts = sorted([f for f in glob.glob(output_path+"/*/*.pth")])
configs = sorted([f for f in glob.glob(output_path+"/*/*.json")])


In [ ]:
 !tts --text "Text for TTS" \
      --model_path "/content/tts_train_dir/run-December-03-2023_05+06PM-0000000/best_model.pth" \
      --config_path "/content/tts_train_dir/run-December-03-2023_05+06PM-0000000/config.json" \
      --out_path out.wav

In [ ]:
import IPython
IPython.display.Audio("out.wav")